## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Vaini,TO,-21.2000,-175.2000,71.76,100,90,5.75,light rain,2021-06-07 04:46:38
1,1,Havelock,US,34.8791,-76.9013,77.95,92,1,1.99,clear sky,2021-06-07 04:46:39
2,2,Mount Isa,AU,-20.7333,139.5000,80.37,26,0,8.05,clear sky,2021-06-07 04:37:54
3,3,Santa Maria Da Boa Vista,BR,-8.8078,-39.8256,69.91,80,90,11.01,overcast clouds,2021-06-07 04:37:55
4,4,Busselton,AU,-33.6500,115.3333,61.05,56,60,9.15,broken clouds,2021-06-07 04:37:55


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Havelock,US,77.95,clear sky,34.8791,-76.9013,
2,Mount Isa,AU,80.37,clear sky,-20.7333,139.5000,
16,Saint-Georges,GD,80.28,few clouds,12.0564,-61.7485,
17,Butaritari,KI,82.40,light rain,3.0707,172.7902,
19,Pattani,TH,87.22,moderate rain,6.7500,100.4167,
20,Antalaha,MG,76.10,scattered clouds,-14.9003,50.2788,
21,Hilo,US,76.28,overcast clouds,19.7297,-155.0900,
28,Korla,CN,83.34,few clouds,41.7597,86.1469,
30,Hiroshima,JP,82.44,overcast clouds,34.3963,132.4594,
34,Teknaf,BD,82.76,overcast clouds,20.8624,92.3058,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        pass

In [9]:
# 7. Drop the rows where there is no Hotel Name.
indexNoHotel = hotel_df[hotel_df["Hotel Name"] == ""].index
hotel_df.drop(indexNoHotel, inplace=True)

In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Havelock,US,77.95,clear sky,34.8791,-76.9013,Sherwood Motel
2,Mount Isa,AU,80.37,clear sky,-20.7333,139.5000,ibis Styles Mt Isa Verona
16,Saint-Georges,GD,80.28,few clouds,12.0564,-61.7485,Radisson Grenada Beach Resort
17,Butaritari,KI,82.40,light rain,3.0707,172.7902,Isles Sunset Lodge
19,Pattani,TH,87.22,moderate rain,6.7500,100.4167,Tobias Kleeberg
20,Antalaha,MG,76.10,scattered clouds,-14.9003,50.2788,Hotel Vitasoa
21,Hilo,US,76.28,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
28,Korla,CN,83.34,few clouds,41.7597,86.1469,Gulisitan Hotel
30,Hiroshima,JP,82.44,overcast clouds,34.3963,132.4594,Oriental Hotel Hiroshima
34,Teknaf,BD,82.76,overcast clouds,20.8624,92.3058,Forestry Guesthouse


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))